<a href="https://colab.research.google.com/github/peculab/AI4JUBO/blob/main/JuboDeath_V5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [121]:
!pip install shap plotly xgboost --quiet

In [122]:
!pip uninstall shap -y
!pip install shap --no-deps

Found existing installation: shap 0.48.0
Uninstalling shap-0.48.0:
  Successfully uninstalled shap-0.48.0
  Using cached shap-0.48.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (25 kB)
Using cached shap-0.48.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.0 MB)


In [123]:
!pip install ace_tools

In [394]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, accuracy_score
from IPython.display import display
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    classification_report, confusion_matrix, mean_absolute_error, r2_score
)

In [395]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [396]:
# read data and put it in a dataframe
# 在 google 工作表載入外部資料 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1MJxCIqPSTQoAnA_Zt9u3FjjvXlWa74pzpC78XDuhQ9o/edit?usp=sharing')
worksheet = gsheets.worksheet("NewTest")  # 指定分頁名稱
worksheet = worksheet.get_all_records()
external = pd.DataFrame(worksheet)
external = external.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
external.head()

,H01_NUM,dbname,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1228,NaN,0,180,0,74,0,0,0,0,0,0.0,0,0.0,1.0,0.0,1.0,1
1,1308,NaN,0,180,1,79,0,0,0,0,0,NaN,0,0.0,1.0,0.0,1.0,1
2,1215,NaN,0,180,0,75,0,0,0,0,0,NaN,0,0.0,1.0,0.0,1.0,1
3,1259,NaN,0,180,1,81,0,0,0,0,0,0.0,0,0.0,1.0,0.0,1.0,1
4,1240,NaN,0,180,1,77,0,0,0,0,0,0.0,0,0.0,1.0,0.0,1.0,1


In [397]:
external['呼吸_ADL低'] = ((external['使用呼吸輔具'] == 1) & (external['ADL_總分_max'] < 30)).astype(int)
external['高齡_住院多'] = ((external['預估年齡'] > 85) & (external['六個月內住院次數'] > 2)).astype(int)
external['ADL低_有管路'] = ((external['ADL_總分_max'] < 30) & (external['使用管路'] == 1)).astype(int)
external['DNR_呼吸'] = ((external['DNR_flag'] == 1) & (external['使用呼吸輔具'] == 1)).astype(int)
external['超級重症'] = (
    (external['DNR_flag'] == 1) &
    (external['使用呼吸輔具'] == 1) &
    (external['ADL_總分_max'] < 30) &
    (external['六個月內住院次數'] > 1)
).astype(int)
external["low_adl_no_var"] = ((external["ADL_std"].isnull()) & (external["ADL_總分_max"] < 5)).astype(int)

In [398]:
external.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,423.0,1412.177305,1484.132874,101.0,1188.0,1222.0,1305.000000,20062.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,423.0,0.404255,0.491328,0.0,0.0,0.0,1.000000,1.000000
觀察天數,423.0,123.595745,70.987948,2.0,43.5,180.0,180.000000,180.000000
性別_is_male,423.0,0.605201,0.489386,0.0,0.0,1.0,1.000000,1.000000
預估年齡,423.0,77.224586,13.404539,21.0,70.0,80.0,87.000000,101.000000
DNR_flag,423.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_總分_max,423.0,16.725768,22.128459,0.0,0.0,5.0,25.000000,100.000000
ADL_first_score,423.0,13.215130,19.998662,0.0,0.0,5.0,20.000000,100.000000
ADL_last_score,423.0,16.725768,22.128459,0.0,0.0,5.0,25.000000,100.000000


In [399]:
ex_missing_info = external.isnull().sum().to_frame(name='Missing Count')
ex_missing_info['Missing Ratio'] = (ex_missing_info['Missing Count'] / len(external)).round(4)
ex_missing_info = ex_missing_info.sort_values(by='Missing Ratio', ascending=True)
ex_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
死亡標記,0,0.0000
觀察天數,0,0.0000
性別_is_male,0,0.0000
DNR_flag,0,0.0000
預估年齡,0,0.0000
ADL_總分_max,0,0.0000
ADL_first_score,0,0.0000
ADL_diff,0,0.0000
ADL_last_score,0,0.0000


In [400]:
# read data and put it in a dataframe
# 在 google 工作表載入訓練資料 gsheets
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1UFokE2ATcNkejRy4PSCkcbCeYzrxOslhelzpd2OaPlc/edit?usp=sharing')
worksheet = gsheets.worksheet("NewDATA")  # 指定分頁名稱
worksheet = worksheet.get_all_records()
df = pd.DataFrame(worksheet)
df = df.apply(lambda col: pd.to_numeric(col.astype(str).str.replace(',', '').str.strip(), errors='coerce'))
df.head()

,H01_NUM,dbname,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,ADL_diff,ADL_std,ADL_明顯惡化,意識清醒_max,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數
0,1340,NaN,0,180,1,92,0,0,0,0,0,0.0,0,0.0,1.0,0.0,1.0,1
1,1181,NaN,0,180,0,83,0,0,0,0,0,NaN,0,0.0,0.0,0.0,1.0,1
2,1272,NaN,0,180,0,85,0,0,0,0,0,NaN,0,0.0,1.0,0.0,1.0,1
3,1245,NaN,0,180,1,86,0,0,0,0,0,0.0,0,0.0,0.0,0.0,1.0,1
4,1390,NaN,0,180,0,87,0,0,0,0,0,0.0,0,0.0,1.0,0.0,1.0,1


In [401]:
df['呼吸_ADL低'] = ((df['使用呼吸輔具'] == 1) & (df['ADL_總分_max'] < 30)).astype(int)
df['高齡_住院多'] = ((df['預估年齡'] > 85) & (df['六個月內住院次數'] > 2)).astype(int)
df['ADL低_有管路'] = ((df['ADL_總分_max'] < 30) & (df['使用管路'] == 1)).astype(int)
df['DNR_呼吸'] = ((df['DNR_flag'] == 1) & (df['使用呼吸輔具'] == 1)).astype(int)
df['超級重症'] = (
    (df['DNR_flag'] == 1) &
    (df['使用呼吸輔具'] == 1) &
    (df['ADL_總分_max'] < 30) &
    (df['六個月內住院次數'] > 1)
).astype(int)
df["low_adl_no_var"] = ((df["ADL_std"].isnull()) & (df["ADL_總分_max"] < 5)).astype(int)

In [402]:
df.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
H01_NUM,15172.0,1420.248286,3241.190305,1.0,1158.0,1225.0,1335.000000,100462.000000
dbname,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
死亡標記,15172.0,0.411613,0.492142,0.0,0.0,0.0,1.000000,1.000000
觀察天數,15172.0,135.797060,61.935242,1.0,82.0,180.0,180.000000,180.000000
性別_is_male,15172.0,0.530385,0.499092,0.0,0.0,1.0,1.000000,1.000000
預估年齡,15172.0,78.423807,12.129807,0.0,72.0,80.0,87.000000,125.000000
DNR_flag,15172.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL_總分_max,15172.0,20.434023,28.977667,0.0,0.0,5.0,30.000000,100.000000
ADL_first_score,15172.0,12.091023,22.469246,0.0,0.0,0.0,15.000000,100.000000
ADL_last_score,15172.0,20.434023,28.977667,0.0,0.0,5.0,30.000000,100.000000


In [403]:
df_missing_info = df.isnull().sum().to_frame(name='Missing Count')
df_missing_info['Missing Ratio'] = (df_missing_info['Missing Count'] / len(df)).round(4)
df_missing_info = df_missing_info.sort_values(by='Missing Ratio', ascending=True)
df_missing_info

,Missing Count,Missing Ratio
H01_NUM,0,0.0000
死亡標記,0,0.0000
觀察天數,0,0.0000
性別_is_male,0,0.0000
DNR_flag,0,0.0000
預估年齡,0,0.0000
ADL_總分_max,0,0.0000
ADL_first_score,0,0.0000
ADL_diff,0,0.0000
ADL_last_score,0,0.0000


In [404]:
features = df_missing_info[df_missing_info['Missing Ratio']<0.5].index.tolist()

In [405]:
features

['H01_NUM',
 '死亡標記',
 '觀察天數',
 '性別_is_male',
 'DNR_flag',
 '預估年齡',
 'ADL_總分_max',
 'ADL_first_score',
 'ADL_diff',
 'ADL_last_score',
 'ADL_明顯惡化',
 '超級重症',
 'ADL低_有管路',
 '高齡_住院多',
 '呼吸_ADL低',
 '六個月內住院次數',
 'low_adl_no_var',
 'DNR_呼吸',
 '使用呼吸輔具',
 '活動假牙',
 '意識清醒_max',
 '使用管路',
 'ADL_std']

In [406]:
dfNew = df[features]

In [407]:
dfNew = dfNew.fillna(0)

In [408]:
from sklearn.base import BaseEstimator, ClassifierMixin

class HybridXGBRF(BaseEstimator, ClassifierMixin):
    def __init__(self, xgb_model=None, rf_model=None, alpha=0.5):
        self.xgb_model = xgb_model
        self.rf_model = rf_model
        self.alpha = alpha
        self._init_models()

    def _init_models(self):
        # Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 4, 'n_estimators': 800, 'subsample': 1.0}
        self.xgb = self.xgb_model or XGBClassifier(
            eval_metric="logloss",
            random_state=42,
            colsample_bytree=0.6,     # ✅ 降低每棵樹看到的特徵比例 → 提高多樣性
            learning_rate=0.02,       # ✅ 稍微提升學習率搭配更早停止
            max_depth=3,              # ✅ 降低單棵樹複雜度 → 降低過擬合
            n_estimators=300,         # ✅ 總樹數可略減以免累積錯誤
            subsample=0.8,            # ✅ 樣本隨機抽樣 → 提升隨機性
            verbosity=0,
            use_label_encoder=False
        )
        self.rf = self.rf_model or RandomForestClassifier(
            n_estimators=300,
            max_depth=5,             # ✅ 控制單棵樹深度，避免全擬合
            max_features='sqrt',     # ✅ 每次切分隨機抽部分特徵
            min_samples_leaf=10,     # ✅ 每個葉節點最少樣本，避免太小造成過擬合
            random_state=42
        )

    def fit(self, X, y):
        self._init_models()  # 每次 fit 要重設模型
        self.xgb.fit(X, y)
        self.rf.fit(X, y)
        return self

    def predict_proba(self, X):
        xgb_prob = self.xgb.predict_proba(X)[:, 1]
        rf_prob = self.rf.predict_proba(X)[:, 1]
        blended = self.alpha * xgb_prob + (1 - self.alpha) * rf_prob
        return np.vstack([1 - blended, blended]).T

    def predict(self, X):
        return (self.predict_proba(X)[:, 1] > 0.5).astype(int)

    def get_params(self, deep=True):
        return {
            'xgb_model': self.xgb_model,
            'rf_model': self.rf_model,
            'alpha': self.alpha
        }

    def set_params(self, **params):
        for param, value in params.items():
            setattr(self, param, value)
        self._init_models()  # 重新初始化模型
        return self

In [409]:
# 要移除的欄位，是代表身分標記，以及天數
drop_columns = ['H01_NUM', '觀察天數']

# 丟掉這些欄位
dfNew = dfNew.drop(columns=drop_columns)

In [410]:
dfNew

,死亡標記,性別_is_male,DNR_flag,預估年齡,ADL_總分_max,ADL_first_score,ADL_diff,ADL_last_score,ADL_明顯惡化,超級重症,...,高齡_住院多,呼吸_ADL低,六個月內住院次數,low_adl_no_var,DNR_呼吸,使用呼吸輔具,活動假牙,意識清醒_max,使用管路,ADL_std
0,0,1,0,92,0,0,0,0,0,0,...,0,1,1,0,0,1.0,0.0,0.0,1.0,0.000000
1,0,0,0,83,0,0,0,0,0,0,...,0,0,1,1,0,0.0,0.0,0.0,1.0,0.000000
2,0,0,0,85,0,0,0,0,0,0,...,0,1,1,1,0,1.0,0.0,0.0,1.0,0.000000
3,0,1,0,86,0,0,0,0,0,0,...,0,0,1,0,0,0.0,0.0,0.0,1.0,0.000000
4,0,0,0,87,0,0,0,0,0,0,...,0,1,1,0,0,1.0,0.0,0.0,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15167,1,1,0,84,50,50,0,50,0,0,...,0,0,1,0,0,1.0,0.0,0.0,1.0,0.000000
15168,1,1,0,84,0,0,0,0,0,0,...,0,1,1,1,0,1.0,0.0,0.0,1.0,0.000000
15169,1,0,0,90,10,10,0,10,0,0,...,0,1,2,0,0,1.0,0.0,0.0,1.0,0.000000
15170,1,1,0,41,5,5,0,5,0,0,...,0,0,1,0,0,0.0,0.0,0.0,1.0,0.000000


In [411]:
import numpy as np
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import (
    confusion_matrix, roc_curve, auc, classification_report
)
import plotly.graph_objects as go
import plotly.express as px

In [412]:
# === 資料準備 ===
X = dfNew.drop(columns=['死亡標記'])
y = df['死亡標記']

In [413]:
X_missing_info = X.isnull().sum().to_frame(name='Missing Count')
X_missing_info['Missing Ratio'] = (X_missing_info['Missing Count'] / len(X)).round(4)
X_missing_info = X_missing_info.sort_values(by='Missing Ratio', ascending=True)
X_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
DNR_flag,0,0.0
預估年齡,0,0.0
ADL_總分_max,0,0.0
ADL_first_score,0,0.0
ADL_diff,0,0.0
ADL_last_score,0,0.0
ADL_明顯惡化,0,0.0
超級重症,0,0.0
ADL低_有管路,0,0.0


# 開始進行訓練

In [414]:
X.describe(include='all').T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,15172.0,0.530385,0.499092,0.0,0.0,1.0,1.000000,1.000000
DNR_flag,15172.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
預估年齡,15172.0,78.423807,12.129807,0.0,72.0,80.0,87.000000,125.000000
ADL_總分_max,15172.0,20.434023,28.977667,0.0,0.0,5.0,30.000000,100.000000
ADL_first_score,15172.0,12.091023,22.469246,0.0,0.0,0.0,15.000000,100.000000
ADL_diff,15172.0,8.343000,18.014530,0.0,0.0,0.0,5.000000,100.000000
ADL_last_score,15172.0,20.434023,28.977667,0.0,0.0,5.0,30.000000,100.000000
ADL_明顯惡化,15172.0,0.239125,0.426563,0.0,0.0,0.0,0.000000,1.000000
超級重症,15172.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
ADL低_有管路,15172.0,0.690812,0.462174,0.0,0.0,1.0,1.000000,1.000000


In [415]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    roc_curve, auc, confusion_matrix, accuracy_score, precision_score,
    recall_score, f1_score, roc_auc_score
)
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import plotly.figure_factory as ff

all_models = {
    "HybridXGBRF (Our Approach)": HybridXGBRF(alpha=1),
    "Alrawi (2013)": LogisticRegression(max_iter=200),
    "Chandra (2022)": XGBClassifier(n_estimators=500, learning_rate=0.01, max_depth=3, random_state=42, eval_metric='logloss'),
    "García-Gollarte (2020)": RandomForestClassifier(n_estimators=100, random_state=42),
    "Levy (2015)": LogisticRegression(max_iter=1000)
}

In [416]:
import copy
from sklearn.base import clone
from sklearn.impute import SimpleImputer
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted

# Prepare CV and ROC Figure
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
fig_roc = go.Figure()
mean_fpr = np.linspace(0, 1, 100)

results = []
trained_models = {}

for model_name, model in all_models.items():
    print(f"▶ Running CV for: {model_name}")
    accs, precs, recalls, f1s, aucs = [], [], [], [], []
    all_cm = np.zeros((2, 2), dtype=int)
    tprs = []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):
        X_train, X_test = X.iloc[train_idx].copy(), X.iloc[test_idx].copy()
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        model_fold = clone(model)

        try:
            # 嘗試直接 fit（若模型支援 NaN，會成功）
            model_fold.fit(X_train, y_train)

        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️ Missing value detected for {model_name} (fold {fold+1}) — applying median imputation.")

                imputer = SimpleImputer(strategy='median')
                X_train = pd.DataFrame(imputer.fit_transform(X_train), columns=X.columns)
                X_test = pd.DataFrame(imputer.transform(X_test), columns=X.columns)

                model_fold.fit(X_train, y_train)
            else:
                raise e  # 若是其他錯誤就直接拋出

        y_pred = model_fold.predict(X_test)
        y_prob = model_fold.predict_proba(X_test)[:, 1]

        # ROC
        fpr, tpr, _ = roc_curve(y_test, y_prob)
        roc_auc = auc(fpr, tpr)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0] = 0.0
        tprs.append(tpr_interp)
        aucs.append(roc_auc)

        # Metrics
        accs.append(accuracy_score(y_test, y_pred))
        precs.append(precision_score(y_test, y_pred))
        recalls.append(recall_score(y_test, y_pred))
        f1s.append(f1_score(y_test, y_pred))
        all_cm += confusion_matrix(y_test, y_pred)

    trained_models[model_name] = copy.deepcopy(model_fold)

    # ROC Curve
    mean_tpr = np.mean(tprs, axis=0)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)

    fig_roc.add_trace(go.Scatter(
        x=mean_fpr, y=mean_tpr, mode='lines',
        name=f"{model_name} (mean AUC={mean_auc:.3f})"
    ))

    results.append({
        'Model': model_name,
        'Accuracy Mean': np.mean(accs),
        'Accuracy Std': np.std(accs),
        'Precision Mean': np.mean(precs),
        'Recall Mean': np.mean(recalls),
        'F1 Score Mean': np.mean(f1s),
        'ROC AUC Mean': np.mean(aucs),
        'ROC AUC Std': np.std(aucs),
        'Confusion Matrix': all_cm
    })

# Add Random Baseline
fig_roc.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1], mode='lines',
    line=dict(dash='dash'), name='Random Baseline'
))

fig_roc.update_layout(
    title="ROC Curve Comparison (Cross-Validation Mean)",
    xaxis_title="False Positive Rate",
    yaxis_title="True Positive Rate",
    width=800, height=600
)

fig_roc.show()

▶ Running CV for: HybridXGBRF (Our Approach)
▶ Running CV for: Alrawi (2013)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

/usr/local/lib/python3.11/dist-packages/sklearn/

▶ Running CV for: Chandra (2022)
▶ Running CV for: García-Gollarte (2020)
▶ Running CV for: Levy (2015)


In [417]:
import plotly.graph_objects as go

for r in results:
    cm = r['Confusion Matrix']
    model_name = r['Model']

    z = cm
    x_labels = ['Predicted 0', 'Predicted 1']
    y_labels = ['Actual 0', 'Actual 1']

    fig_cm = go.Figure(data=go.Heatmap(
        z=z,
        x=x_labels,
        y=y_labels,
        colorscale='Blues',
        text=z,
        texttemplate="%{text}"
    ))

    fig_cm.update_layout(
        title=f"Confusion Matrix - {model_name}",
        xaxis_title="Predicted Label",
        yaxis_title="True Label",
        width=500,
        height=500
    )

    fig_cm.show()

In [418]:
# Convert results to DataFrame
results_df = pd.DataFrame(results)
results_df = results_df.sort_values(by="ROC AUC Mean", ascending=False).reset_index(drop=True)
results_df

,Model,Accuracy Mean,Accuracy Std,Precision Mean,Recall Mean,F1 Score Mean,ROC AUC Mean,ROC AUC Std,Confusion Matrix
0,HybridXGBRF (Our Approach),0.765159,0.007674,0.750532,0.643395,0.692772,0.839553,0.006766,"[[7591, 1336], [2227, 4018]]"
1,Chandra (2022),0.765621,0.006365,0.750084,0.645957,0.694079,0.838746,0.005985,"[[7582, 1345], [2211, 4034]]"
2,Levy (2015),0.729765,0.004329,0.695581,0.610889,0.650479,0.805616,0.006744,"[[7257, 1670], [2430, 3815]]"
3,García-Gollarte (2020),0.743540,0.005424,0.703071,0.652682,0.676867,0.805078,0.006668,"[[7205, 1722], [2169, 4076]]"
4,Alrawi (2013),0.727920,0.005588,0.689663,0.616653,0.651070,0.804456,0.006567,"[[7193, 1734], [2394, 3851]]"


# 測試外部資料在 XGBoost 模型下的結果

In [419]:
external.head()

,H01_NUM,dbname,死亡標記,觀察天數,性別_is_male,預估年齡,DNR_flag,ADL_總分_max,ADL_first_score,ADL_last_score,...,使用呼吸輔具,活動假牙,使用管路,六個月內住院次數,呼吸_ADL低,高齡_住院多,ADL低_有管路,DNR_呼吸,超級重症,low_adl_no_var
0,1228,NaN,0,180,0,74,0,0,0,0,...,1.0,0.0,1.0,1,1,0,1,0,0,0
1,1308,NaN,0,180,1,79,0,0,0,0,...,1.0,0.0,1.0,1,1,0,1,0,0,1
2,1215,NaN,0,180,0,75,0,0,0,0,...,1.0,0.0,1.0,1,1,0,1,0,0,1
3,1259,NaN,0,180,1,81,0,0,0,0,...,1.0,0.0,1.0,1,1,0,1,0,0,0
4,1240,NaN,0,180,1,77,0,0,0,0,...,1.0,0.0,1.0,1,1,0,1,0,0,0


In [420]:
ex_X = external[features].drop(columns=['死亡標記'])
ex_y = external['死亡標記']

In [421]:
ex_X = ex_X.fillna(0)

In [422]:
# 丟掉這些欄位
ex_X = ex_X.drop(columns=drop_columns)

In [423]:
ex_X.describe().T

,count,mean,std,min,25%,50%,75%,max
性別_is_male,423.0,0.605201,0.489386,0.0,0.0,1.0,1.0,1.000000
DNR_flag,423.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
預估年齡,423.0,77.224586,13.404539,21.0,70.0,80.0,87.0,101.000000
ADL_總分_max,423.0,16.725768,22.128459,0.0,0.0,5.0,25.0,100.000000
ADL_first_score,423.0,13.215130,19.998662,0.0,0.0,5.0,20.0,100.000000
ADL_diff,423.0,3.510638,10.119160,0.0,0.0,0.0,0.0,100.000000
ADL_last_score,423.0,16.725768,22.128459,0.0,0.0,5.0,25.0,100.000000
ADL_明顯惡化,423.0,0.139480,0.346857,0.0,0.0,0.0,0.0,1.000000
超級重症,423.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000
ADL低_有管路,423.0,0.711584,0.453562,0.0,0.0,1.0,1.0,1.000000


In [424]:
eX_missing_info = ex_X.isnull().sum().to_frame(name='Missing Count')
eX_missing_info['Missing Ratio'] = (eX_missing_info['Missing Count'] / len(ex_X)).round(4)
eX_missing_info = eX_missing_info.sort_values(by='Missing Ratio', ascending=False)
eX_missing_info

,Missing Count,Missing Ratio
性別_is_male,0,0.0
DNR_flag,0,0.0
預估年齡,0,0.0
ADL_總分_max,0,0.0
ADL_first_score,0,0.0
ADL_diff,0,0.0
ADL_last_score,0,0.0
ADL_明顯惡化,0,0.0
超級重症,0,0.0
ADL低_有管路,0,0.0


In [425]:
from sklearn.impute import SimpleImputer
from sklearn.base import clone
from sklearn.exceptions import NotFittedError
from sklearn.utils.validation import check_is_fitted
import plotly.subplots as sp

def evaluate_all_models_visual(models: dict, X_val, y_val):
    mean_fpr = np.linspace(0, 1, 100)
    fig_roc = go.Figure()
    results = []

    # 建立混淆矩陣子圖
    num_models = len(models)
    cols = 3
    rows = int(np.ceil(num_models / cols))

    fig_cm = sp.make_subplots(
        rows=rows, cols=cols,
        subplot_titles=list(models.keys()),
        horizontal_spacing=0.15,
        vertical_spacing=0.15
    )

    for i, (model_name, model) in enumerate(models.items()):
        print(f"🔍 Evaluating {model_name}...")

        # 嘗試使用原始資料
        X_input = X_val.copy()
        y_input = y_val

        # 若模型不支援 NaN，則補值
        try:
            # 嘗試呼叫 predict_proba
            _ = model.predict_proba(X_input)
        except ValueError as e:
            if "Input X contains NaN" in str(e):
                print(f"⚠️  {model_name} 不支援 NaN，自動補值中...")
                imputer = SimpleImputer(strategy="median")
                X_input = pd.DataFrame(imputer.fit_transform(X_input), columns=X_val.columns)
            else:
                raise e

        # 預測
        y_pred = model.predict(X_input)
        y_prob = model.predict_proba(X_input)[:, 1]

        # 指標
        acc = accuracy_score(y_input, y_pred)
        prec = precision_score(y_input, y_pred)
        rec = recall_score(y_input, y_pred)
        f1 = f1_score(y_input, y_pred)
        auc_val = roc_auc_score(y_input, y_prob)

        # ROC
        fpr, tpr, _ = roc_curve(y_input, y_prob)
        tpr_interp = np.interp(mean_fpr, fpr, tpr)
        tpr_interp[0], tpr_interp[-1] = 0.0, 1.0

        fig_roc.add_trace(go.Scatter(
            x=mean_fpr, y=tpr_interp,
            mode='lines',
            name=f"{model_name} (AUC={auc_val:.3f})"
        ))

        # 混淆矩陣
        cm = confusion_matrix(y_input, y_pred)
        row, col = i // cols + 1, i % cols + 1
        fig_cm.add_trace(
            go.Heatmap(
                z=cm,
                x=["Predicted Negative", "Predicted Positive"],
                y=["Actual Negative", "Actual Positive"],
                colorscale='Blues',
                text=cm,
                texttemplate="%{text}",
                showscale=False
            ),
            row=row, col=col
        )

        results.append({
            "Model": model_name,
            "Accuracy": acc,
            "Precision": prec,
            "Recall": rec,
            "F1 Score": f1,
            "ROC AUC": auc_val
        })

    # 隨機基準線
    fig_roc.add_trace(go.Scatter(
        x=[0, 1], y=[0, 1],
        mode='lines',
        line=dict(dash='dash'),
        name='Random Baseline'
    ))

    fig_roc.update_layout(
        title="ROC Curve Comparison",
        xaxis_title="False Positive Rate",
        yaxis_title="True Positive Rate",
        width=800,
        height=600
    )
    fig_roc.show()

    fig_cm.update_layout(
        title="Confusion Matrices of All Models",
        width=400 * cols,
        height=300 * rows,
        showlegend=False
    )
    fig_cm.show()

    # 指標表格
    df_result = pd.DataFrame(results)

    return df_result

In [426]:
# 假設已經訓練完模型並存在 trained_models 中
evaluate_all_models_visual(trained_models, ex_X, ex_y)

🔍 Evaluating HybridXGBRF (Our Approach)...
🔍 Evaluating Alrawi (2013)...
🔍 Evaluating Chandra (2022)...
🔍 Evaluating García-Gollarte (2020)...
🔍 Evaluating Levy (2015)...


,Model,Accuracy,Precision,Recall,F1 Score,ROC AUC
0,HybridXGBRF (Our Approach),0.661939,0.564815,0.713450,0.630491,0.704957
1,Alrawi (2013),0.652482,0.581081,0.502924,0.539185,0.666040
2,Chandra (2022),0.666667,0.572115,0.695906,0.627968,0.702439
3,García-Gollarte (2020),0.619385,0.523364,0.654971,0.581818,0.664033
4,Levy (2015),0.652482,0.580000,0.508772,0.542056,0.670403
